## Environment Setting

In [1]:
import os
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## Utility Function

In [ ]:
import numpy as np
import imageio


def make_puzzle(imgs, row, col, path=None):
    h, w, c = imgs[0].shape
    out = np.zeros((h * row, w * col, c), np.uint8)
    for n, img in enumerate(imgs):
        j, i = divmod(n, col)
        out[j * h: (j + 1) * h, i * w: (i + 1) * w, :] = img
    if path is not None:
        imageio.imwrite(path, out)
    return out


## Parameter Setting

In [ ]:
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64
IMAGE_CHANNEL = 3

SAMPLE_ROW = 8
SAMPLE_COL = 8
SAMPLE_NUM = SAMPLE_ROW * SAMPLE_COL

hparams = {
    'HIDDEN_DIM': 64,
    'DENSE_DIM': 128,
    'Z_DIM': 100,
    'BATCH_SIZE': 128,
    'LEARNING_RATE': 1e-4,
    'BETA_1': 0.5,
    'BETA_2': 0.999,
    'LAMBDA': 10,
    'NUM_EPOCH': 100,
    'NOISE_DECAY_LIMIT': 30,
    'CHECKPOINTS_DIR': './checkpoints/ckpt/',
    'TRAIN_FROM_LATEST_EPOCH': False,
    'PRINT_FREQ': 10
}


## Preprocess Training Data

In [ ]:
dictionary_dir = './data/dictionary/'
word2Id_dict = dict(np.load(dictionary_dir + 'word2Id.npy'))
id2word_dict = dict(np.load(dictionary_dir + 'id2Word.npy'))


def indices_list_to_text_list(indices_list):
    text_list = []
    for indices in indices_list:
        word_list = []
        for idx in indices:
            if idx == word2Id_dict['<RARE>']:
                continue
            if idx == word2Id_dict['<PAD>']:
                break
            word_list.append(id2word_dict[idx])
        text = ' '.join(word_list)
        if len(text.strip()) != 0:
            text_list.append(text)
    return text_list


### Text Embedding with Sentence-BERT

Install `SentenceTransformer`
```sh
$ conda install -c conda-forge sentence-transformers
$ conda install -c conda-forge ipywidgets
```

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

dataset_dir = './data/dataset/'
if os.path.exists(dataset_dir + 'train_data_embedding.pkl'):
    print("if")
    df_train = pd.read_pickle(dataset_dir + 'train_data_embedding.pkl')
    df_train.head()
else:
    print("el")
    sbert = SentenceTransformer('all-mpnet-base-v2')
    df_train = pd.read_pickle(dataset_dir + 'text2ImgData.pkl')
    df_train['Texts'] = df_train['Captions'].apply(lambda x: indices_list_to_text_list(x))
    df_train['Embeddings'] = df_train['Texts'].apply(lambda x: sbert.encode(x))
    df_train.head()
    df_train.to_pickle(dataset_dir + 'train_data_embedding.pkl')


/home/wesley/sclab-nas_home/anaconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


el


In [ ]:
!pip list


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Package                       Version
----------------------------- ------------
absl-py                       2.0.0
albumentations                1.3.1
asttokens                     2.4.1
astunparse                    1.6.3
backports.functools-lru-cache 1.6.5
cachetools                    5.3.2
certifi                       2023.7.22
cffi                          1.16.0
charset-normalizer            3.3.2
clang                         5.0
click                         8.1.7
colorama                      0.4.6
comm                          0.1.4
cryptography                  41.0.5
cycler                        0.12.1
debugpy                       1.6.7
decorator                     4.4.2
defusedxml                    0.7.1
entrypoints                   0.4
exceptiongroup                1.1.3
executing                     2.0.1
filelock                      3.13.1
flatbuffers                   23.5.26
fsspec                        2023.12.2
gast                          0.4.0
Glances  

In [ ]:
df_train.head()


,Captions,ImagePath,Texts,Embeddings
ID,,,,
6734,"[[9, 2, 17, 9, 1, 6, 14, 13, 18, 3, 41, 8, 11,...",./102flowers/image_06734.jpg,[the petals of the flower are pink in color an...,"[[0.049325254, -0.08779175, -0.04624424, 0.074..."
6736,"[[4, 1, 5, 12, 2, 3, 11, 31, 28, 68, 106, 132,...",./102flowers/image_06736.jpg,[this flower has white petals and yellow pisti...,"[[-0.022878254, -0.02727319, -0.019405542, 0.0..."
6737,"[[9, 2, 27, 4, 1, 6, 14, 7, 12, 19, 5427, 5427...",./102flowers/image_06737.jpg,[the petals on this flower are pink with white...,"[[0.004260789, -0.02295248, -0.04379613, 0.040..."
6738,"[[9, 1, 5, 8, 54, 16, 38, 7, 12, 116, 325, 3, ...",./102flowers/image_06738.jpg,[the flower has a smooth purple petal with whi...,"[[0.035809662, -0.020764334, -0.03761639, 0.06..."
6739,"[[4, 12, 1, 5, 29, 11, 19, 7, 26, 70, 5427, 54...",./102flowers/image_06739.jpg,[this white flower has bright yellow stamen wi...,"[[-0.014632274, -0.072241016, -0.046013944, 0...."


## Create Training Dataset

In [ ]:
import random


def map_train(embedding, image_path):
    embedding = tf.cast(embedding, tf.float32)

    img = tf.io.read_file(image_path)
    img = tf.image.decode_image(img, channels=IMAGE_CHANNEL)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img.set_shape([None, None, IMAGE_CHANNEL])
    img = tf.image.resize(img, size=[IMAGE_HEIGHT, IMAGE_WIDTH])
    img.set_shape([IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNEL])

    if tf.random.uniform([]) < 0.25:
        img = tf.image.flip_left_right(img)
    if tf.random.uniform([]) < 0.25:
        img = tf.image.random_brightness(img, max_delta=0.2)
    img = img * 2 - 1

    return embedding, img


def train_dataset_generator(embedding_array, image_path_array, batch_size):
    embedding_list = []
    image_path_list = []
    for embeddings, image_path in zip(embedding_array, image_path_array):
        for embedding in embeddings:
            embedding_list.append(embedding)
            image_path_list.append(image_path)

    dataset = tf.data.Dataset.from_tensor_slices((embedding_list, image_path_list))
    dataset = dataset.map(map_train, tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(len(embedding_list))
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return dataset


In [ ]:
dataset_train = train_dataset_generator(df_train['Embeddings'].values, df_train['ImagePath'].values, hparams['BATCH_SIZE'])
print(f'There are {len(dataset_train) * hparams["BATCH_SIZE"]:d} samples in training data.')


There are 70400 samples in training data.


## Conditional GAN Model (DCGAN)

In [ ]:
conv_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)
batchnorm_init = tf.keras.initializers.RandomNormal(mean=1.0, stddev=0.02)


### Generator

In [ ]:
def deconv_batchnorm_relu(filters):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2DTranspose(
            filters=filters,
            kernel_size=5,
            strides=2,
            padding='SAME',
            output_padding=1,
            use_bias=False,
            kernel_initializer=conv_init
        ),
        tf.keras.layers.BatchNormalization(gamma_initializer=batchnorm_init),
        tf.keras.layers.ReLU()
    ])


class Generator(tf.keras.Model):
    def __init__(self, hparams):
        super(Generator, self).__init__()
        self.hparams = hparams
        self.dim = self.hparams['HIDDEN_DIM']

        self.embedding_layers = tf.keras.Sequential([
            tf.keras.layers.Dense(units=self.hparams['DENSE_DIM']),
            tf.keras.layers.LeakyReLU(0.2)
        ])

        self.concat_layers = tf.keras.Sequential([
            tf.keras.layers.Dense(units=self.dim * 8 * 4 * 4, use_bias=False),
            tf.keras.layers.BatchNormalization(gamma_initializer=batchnorm_init),
            tf.keras.layers.ReLU(),

            tf.keras.layers.Reshape([4, 4, -1]),

            deconv_batchnorm_relu(self.dim * 4),
            deconv_batchnorm_relu(self.dim * 2),
            deconv_batchnorm_relu(self.dim * 1),
            tf.keras.layers.Conv2DTranspose(
                filters=IMAGE_CHANNEL,
                kernel_size=5,
                strides=2,
                padding='SAME',
                output_padding=1,
                activation=tf.keras.activations.tanh,
                kernel_initializer=conv_init
            )
        ])

    def call(self, embedding, noise_z):
        embedding = self.embedding_layers(embedding)

        concat = tf.concat([noise_z, embedding], axis=1)

        output = self.concat_layers(concat)

        return output


### Discriminator

In [ ]:
def conv_batchnorm_leaky_relu(filters):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(
            filters=filters,
            kernel_size=5,
            strides=2,
            padding='SAME',
            kernel_initializer=conv_init
        ),
        tf.keras.layers.BatchNormalization(gamma_initializer=batchnorm_init),
        tf.keras.layers.LeakyReLU(0.2)
    ])


class Discriminator(tf.keras.Model):
    def __init__(self, hparams):
        super(Discriminator, self).__init__()
        self.hparams = hparams
        self.dim = self.hparams['HIDDEN_DIM']

        self.embedding_layers = tf.keras.Sequential([
            tf.keras.layers.Dense(units=self.hparams['DENSE_DIM']),
            tf.keras.layers.LeakyReLU(0.2)
        ])

        self.image_layers = tf.keras.Sequential([
            tf.keras.layers.Conv2D(
                filters=self.dim,
                kernel_size=5,
                strides=2,
                padding='SAME',
                kernel_initializer=conv_init
            ),
            tf.keras.layers.LeakyReLU(0.2),
            conv_batchnorm_leaky_relu(self.dim * 2),
            conv_batchnorm_leaky_relu(self.dim * 4),
            conv_batchnorm_leaky_relu(self.dim * 8)
        ])

        self.concat_layers = tf.keras.Sequential([
            tf.keras.layers.Conv2D(
                filters=self.dim * 8,
                kernel_size=1,
                strides=1,
                padding='SAME',
                kernel_initializer=conv_init
            ),
            tf.keras.layers.LeakyReLU(0.2),
            tf.keras.layers.Conv2D(
                filters=1,
                kernel_size=4,
                strides=1,
                padding='VALID',
                kernel_initializer=conv_init
            ),
            tf.keras.layers.Reshape([])
        ])

    def call(self, embedding, image):
        embedding = self.embedding_layers(embedding)
        embedding = tf.expand_dims(embedding, axis=1)
        embedding = tf.expand_dims(embedding, axis=1)
        embedding = tf.tile(embedding, multiples=[1, 4, 4, 1])

        image = self.image_layers(image)

        concat = tf.concat([image, embedding], axis=-1)

        output = self.concat_layers(concat)

        return output


In [ ]:
generator = Generator(hparams)
discriminator = Discriminator(hparams)


### Optimizer and Loss Function

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(hparams['LEARNING_RATE'], hparams['BETA_1'], hparams['BETA_2'])
discriminator_optimizer = tf.keras.optimizers.Adam(hparams['LEARNING_RATE'], hparams['BETA_1'], hparams['BETA_2'])


In [ ]:
@tf.function
def G_train_step(embedding, real_data, noise_decay):
    noise_z = tf.random.normal([hparams['BATCH_SIZE'], hparams['Z_DIM']])

    with tf.GradientTape() as tape:
        fake_data = generator(embedding, noise_z, training=True)
        fake_data = fake_data + noise_decay * tf.random.normal(fake_data.shape)
        real_data = real_data + noise_decay * tf.random.normal(real_data.shape)

        D_fake = discriminator(embedding, fake_data, training=True)
        D_real = discriminator(embedding, real_data, training=True)

        loss_g = tf.reduce_mean(-D_fake)

        epsilon = tf.random.uniform([hparams['BATCH_SIZE'], 1, 1, 1])
        interpolates = epsilon * real_data + (1 - epsilon) * fake_data
        interpolates = interpolates + noise_decay * tf.random.normal(interpolates.shape)
        slopes = tf.gradients(discriminator(embedding, interpolates, training=True), interpolates)[0]
        l2_norm = tf.sqrt(tf.reduce_sum(slopes ** 2, axis=[1, 2, 3]))
        gradient_penalty = (l2_norm - 1.) ** 2

        loss_d = tf.reduce_mean(D_fake - D_real + hparams['LAMBDA'] * gradient_penalty)

    gradient = tape.gradient(loss_g, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradient, generator.trainable_variables))

    return loss_g, loss_d


@tf.function
def D_train_step(embedding, real_data, noise_decay):
    noise_z = tf.random.normal([hparams['BATCH_SIZE'], hparams['Z_DIM']])

    with tf.GradientTape() as tape:
        fake_data = generator(embedding, noise_z, training=True)
        fake_data = fake_data + noise_decay * tf.random.normal(fake_data.shape)
        real_data = real_data + noise_decay * tf.random.normal(real_data.shape)

        D_fake = discriminator(embedding, fake_data, training=True)
        D_real = discriminator(embedding, real_data, training=True)

        loss_g = tf.reduce_mean(-D_fake)

        epsilon = tf.random.uniform([hparams['BATCH_SIZE'], 1, 1, 1])
        interpolates = epsilon * real_data + (1 - epsilon) * fake_data
        interpolates = interpolates + noise_decay * tf.random.normal(interpolates.shape)
        slopes = tf.gradients(discriminator(embedding, interpolates, training=True), interpolates)[0]
        l2_norm = tf.sqrt(tf.reduce_sum(slopes ** 2, axis=[1, 2, 3]))
        gradient_penalty = (l2_norm - 1.) ** 2

        loss_d = tf.reduce_mean(D_fake - D_real + hparams['LAMBDA'] * gradient_penalty)

    gradient = tape.gradient(loss_d, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradient, discriminator.trainable_variables))

    return loss_g, loss_d


In [ ]:
train_step = (
    D_train_step,
    D_train_step,
    G_train_step
)

num_critic = len(train_step)


## Choose Training Sample for Validation

In [ ]:
samples_dir = './samples/'
if not os.path.exists(samples_dir):
    os.makedirs(samples_dir)


In [ ]:
sample_embeddings = []
for embeddings in df_train['Embeddings'].values:
    if len(embeddings) >= SAMPLE_ROW:
        for j in range(SAMPLE_ROW):
            sample_embeddings.append(embeddings[j])
    if len(sample_embeddings) == SAMPLE_NUM:
        break

sample_embeddings = tf.Variable(sample_embeddings)
print(sample_embeddings.shape)


(64, 768)


## Training

In [ ]:
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)
checkpoint_manager = tf.train.CheckpointManager(checkpoint, hparams['CHECKPOINTS_DIR'], max_to_keep=20)

start_epoch = 0
if hparams['TRAIN_FROM_LATEST_EPOCH'] and checkpoint_manager.latest_checkpoint:
    start_epoch = int(checkpoint_manager.latest_checkpoint.split('-')[-1])
    checkpoint.restore(checkpoint_manager.latest_checkpoint)


In [ ]:
from tqdm import trange
import numpy as np
import matplotlib.pyplot as plt


def train(start_epoch, stop_epoch):
    g_loss_list = [None] * (stop_epoch - start_epoch)
    d_loss_list = [None] * (stop_epoch - start_epoch)

    critic = 0
    num_train_step = len(dataset_train)
    sample_noise = tf.random.normal([SAMPLE_NUM, hparams['Z_DIM']])

    pbar = trange(start_epoch, stop_epoch, unit='epoch')
    for epoch in pbar:
        loss_g_t = 0.0
        loss_d_t = 0.0

        if epoch < hparams['NOISE_DECAY_LIMIT']:
            noise_decay = 1 / (epoch + 1)
        else:
            noise_decay = 0

        for embedding, real_data in dataset_train:
            loss_g, loss_d = train_step[critic](embedding, real_data, noise_decay)
            critic = critic + 1 if critic + 1 < num_critic else 0
            loss_g_t += loss_g.numpy()
            loss_d_t += loss_d.numpy()

        g_loss_list[epoch] = loss_g_t / num_train_step
        d_loss_list[epoch] = loss_d_t / num_train_step
        pbar.set_postfix({'g_loss': loss_g_t / num_train_step, 'd_loss': loss_d_t / num_train_step})

        out = generator(sample_embeddings, sample_noise, training=False)
        img = make_puzzle(
            ((out + 1) / 2 * 255).numpy().astype(np.uint8),
            SAMPLE_ROW,
            SAMPLE_COL,
            f'{samples_dir}train_{epoch + 1:04d}.png'
        )
        if (epoch + 1) % hparams['PRINT_FREQ'] == 0:
            plt.imshow(img)
            plt.axis('off')
            plt.title(f'Epoch {epoch + 1:d}')
            plt.show()

        checkpoint_manager.save(epoch + 1)

    return g_loss_list, d_loss_list


In [ ]:
!pip install tensorflow==2.6


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.4/458.4 MB 1.5 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 22.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  You can safely remove it manually.
  You can safely remove it manually.
  Attempting uninstall: absl-py
    Found existing installation: absl-py 2.0.0
    Uninstalling absl-py-2.0.0:
      Successfully uninstalled absl-py-2.0.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
      Successfully uninstalled tensorflow-2.8.0
 

In [ ]:
g_loss_list, d_loss_list = train(start_epoch, hparams['NUM_EPOCH'])


  0%|          | 0/100 [00:00<?, ?epoch/s]2023-12-16 23:25:40.113587: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 11187 of 70495
2023-12-16 23:25:50.112374: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 22540 of 70495
2023-12-16 23:26:00.114399: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 34559 of 70495
2023-12-16 23:26:13.877666: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 46469 of 70495
2023-12-16 23:26:30.122094: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 67415 of 70495
2023-12-16 23:26:33.052305: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buff

UnimplementedError: Graph execution error:

Detected at node discriminator/sequential_9/conv2d/Conv2D_1 defined at (most recent call last):
  File "/home/wesley/sclab-nas_home/anaconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/wesley/sclab-nas_home/anaconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/wesley/sclab-nas_home/anaconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/wesley/sclab-nas_home/anaconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1053, in launch_instance

  File "/home/wesley/sclab-nas_home/anaconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 737, in start

  File "/home/wesley/sclab-nas_home/anaconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start

  File "/home/wesley/sclab-nas_home/anaconda3/envs/tf/lib/python3.9/asyncio/base_events.py", line 601, in run_f

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(start_epoch, hparams['NUM_EPOCH']), g_loss_list, color='red', label='Generator Loss')
plt.plot(range(start_epoch, hparams['NUM_EPOCH']), d_loss_list, color='blue', label='Discriminator Loss')
plt.legend(loc='upper right')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('WGAN-GP Training Loss')
plt.tight_layout()
plt.show()


: 

# Evaluation

## Preprocess Testing Data

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

if os.path.exists(dataset_dir + 'test_data_embedding.pkl'):
    df_test = pd.read_pickle(dataset_dir + 'test_data_embedding.pkl')
else:
    sbert = SentenceTransformer('all-mpnet-base-v2')
    df_test = pd.read_pickle(dataset_dir + 'testData.pkl')
    df_test['Texts'] = df_test['Captions'].apply(lambda x: indices_list_to_text_list([x]))
    df_test['Embeddings'] = df_test['Texts'].apply(lambda x: sbert.encode(x)[0])
    df_test.to_pickle(dataset_dir + 'test_data_embedding.pkl')


: 

In [ ]:
df_test.head()


: 

## Create Testing Dataset

In [ ]:
def map_test(embedding, index):
    embedding = tf.cast(embedding, tf.float32)
    return embedding, index


def test_dataset_generator(embedding_array, index_array, batch_size):
    embedding_list = []
    index_list = []
    for embedding, index in zip(embedding_array, index_array):
        embedding_list.append(embedding)
        index_list.append(index)

    dataset = tf.data.Dataset.from_tensor_slices((embedding_list, index_list))
    dataset = dataset.map(map_test, tf.data.experimental.AUTOTUNE)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size, drop_remainder=True)

    return dataset


: 

In [ ]:
dataset_test = test_dataset_generator(df_test['Embeddings'].values, df_test['ID'].values, hparams['BATCH_SIZE'])
print(f'There are {len(df_test):d} samples in testing dataset.')


: 

## Inference

In [ ]:
inference_dir = './inference/'
if not os.path.exists(inference_dir):
    os.makedirs(inference_dir)


: 

In [ ]:
@tf.function
def test_step(embedding, noise):
    fake_image = generator(embedding, noise, training=False)
    return fake_image


: 

In [ ]:
checkpoint.restore(checkpoint_manager.latest_checkpoint)


: 

In [ ]:
import time


def inference():
    num_test_step = len(df_test) // hparams['BATCH_SIZE']
    sample_noise = np.random.normal(loc=0.0, scale=1.0, size=(hparams['Z_DIM'])).astype(np.float32)
    sample_noise = np.tile(sample_noise, (hparams['BATCH_SIZE'], 1))

    step = 0
    start = time.time()
    for embedding, index in dataset_test:
        fake_image = test_step(embedding, sample_noise)
        for i in range(hparams['BATCH_SIZE']):
            plt.imsave(f'{inference_dir}inference_{index[i]:04d}.jpg', (fake_image[i].numpy() + 1) / 2)

        step += 1
        if step > num_test_step:
            break

    print('Time for inference is {:.4f} sec'.format(time.time()-start))


: 

In [ ]:
inference()


: 

## Inception Score & Cosine Similarity

**Clear GPU memory to avoid OOM!**

Install `numba`
```sh
$ conda install -c numba numba
```

In [ ]:
from numba import cuda

cuda.get_current_device().reset()


: 

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
score_file = './score.csv'
if os.path.exists(score_file):
    os.remove(score_file)


: 

In [ ]:
%cd testing
!python inception_score_debug.py ../inference/ ../score.csv 39
%cd ..


: 

In [ ]:
import os
import pandas as pd
import numpy as np

if os.path.exists(score_file):
    df_score = pd.read_csv(score_file)
    mean_score = np.mean(df_score['score'].values)
    print(f'Mean Score: {mean_score:f}')
else:
    print('Evaluation Failed!')


: 